In [ ]:
from finance_utils.electrical.china.spot.discrete.probabilistic_forecast import DiscreteSpot, \
    ProbabilisticDiscreteCurve, SpotNoise
from finance_utils.electrical.china.spot.discrete.testback import TestBack
from finance_utils.electrical.china.spot.rule.recycle import SampleRecycle
import numpy
from data_utils.stochastic_utils.distributions.baseclass import ABCDistribution
from data_utils.stochastic_utils.distributions.basic_distributions import NormalDistribution
from data_utils.stochastic_utils.distributions.nonParametricDistribution import HistogramDist, LogHisDist
from data_utils.stochastic_utils.distributions.discrete.basic import DiscreteDistribution
from matplotlib import pyplot
import tqdm
import json

from easy_utils.db_utils.mysql import MySQLLink
from easy_utils.obj_utils.enumerable_utils import flatten
from private.db import mysql_94j6daxc

In [ ]:
quantity_min = 0
quantity_max = 30
submitted_min = 0
submitted_max = 30
trigger_rate = 0.05
punishment_rate = 0.5
random_p = 0
random_p_2 = 0.5

In [ ]:
db = MySQLLink(
    url=mysql_94j6daxc["url"],
    port=mysql_94j6daxc["port"],
    user=mysql_94j6daxc["user"],
    pwd=mysql_94j6daxc["pwd"],
    db="tianrun_etrade"
)

In [ ]:
d,_,__ = db.select(
    "select time_order,dayahead_statement_price,realtime_statement_price,dayahead_clearing_quantity,realtime_clearing_quantity "
    "from spot_real "
    "where station_id='14013006' and date between '2024-5-1' and '2024-8-1'"
)
a= numpy.array(d)

d2,_,__ = db.select(
    "select time_order,dayahead_statement_price,realtime_statement_price,dayahead_clearing_quantity,realtime_clearing_quantity "
    "from spot_real "
    "where station_id='14013006' and date between '2024-8-1' and '2024-9-1'"
)
a2= numpy.array(d2)

In [ ]:
def get_slice(xlist, order, slice):
    return xlist[numpy.where(xlist[:,0]==order)][:,slice]

In [ ]:
dp_his_list = []
rp_his_list = []
aq_his_list = []
max_aq = []
sq_list = []
for i in range(1,5):
    dp = get_slice(a,i,1)
    rp = get_slice(a,i,2)
    sq = get_slice(a,i,3)
    rq = get_slice(a,i,4)
    aq = sq + rq
    dp_his_list.append(HistogramDist(dp))
    rp_his_list.append(HistogramDist(rp))
    aq_his_list.append(HistogramDist(aq))
    max_aq.append(numpy.max(aq))
    
dp_his_curve = ProbabilisticDiscreteCurve(dp_his_list)
rp_his_curve = ProbabilisticDiscreteCurve(rp_his_list)
aq_his_curve = ProbabilisticDiscreteCurve(aq_his_list, domain_min=0, domain_max=numpy.max(max_aq))

In [15]:
spot = DiscreteSpot(
    dp_his_curve,
    rp_his_curve,
    aq_his_curve
)
print(spot.quantity.value_list)

[5.560247300688938, 5.480086005731937, 5.484021503235704, 5.468258067320752]


In [16]:
print(spot.differential_evolution__search(
    submitted_min=0,
    submitted_max=numpy.max(max_aq),
    trigger_rate = trigger_rate,
    punishment_rate = punishment_rate
))

(array([20.4053772 ,  0.30671945,  0.13238277,  0.08996178]), 7729.331228407268)
